In [2]:
%load_ext autoreload
%autoreload 2

from typing import Optional

import torch
from botorch.models import FixedNoiseGP
from botorch.optim.fit import fit_gpytorch_scipy
from botorch.fit import fit_fully_bayesian_model_nuts
from botorch.test_functions import Branin, Hartmann
from gpytorch.kernels import Kernel, MaternKernel, ScaleKernel
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.priors import HalfCauchyPrior

In [3]:
tkwargs = {
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "dtype": torch.double,
}

In [4]:
class SaasPriorHelper:
    """Helper class for specifying parameter and setting closures."""

    def __init__(self, tau: Optional[float] = None):
        self._tau = tau

    def tau(self, m):
        return (
            self._tau
            if self._tau is not None
            else m.raw_tau_constraint.transform(m.raw_tau)
        )

    def inv_lengthscale_prior_param_or_closure(self, m):
        return self.tau(m) / (m.lengthscale ** 2)

    def inv_lengthscale_prior_setting_closure(self, m, value):
        lb = m.raw_lengthscale_constraint.lower_bound
        ub = m.raw_lengthscale_constraint.upper_bound
        m._set_lengthscale((self.tau(m) / value).sqrt().clamp(lb, ub))

    def tau_prior_param_or_closure(self, m):
        return m.raw_tau_constraint.transform(m.raw_tau)

    def tau_prior_setting_closure(self, m, value):
        lb = m.raw_tau_constraint.lower_bound
        ub = m.raw_tau_constraint.upper_bound
        m.raw_tau.data.fill_(
            m.raw_tau_constraint.inverse_transform(value.clamp(lb, ub)).item()
        )

In [5]:
def add_saas_prior(
    base_kernel: Kernel, tau: Optional[float] = 0.1, **tkwargs
) -> Kernel:
    if not base_kernel.has_lengthscale:
        raise UnsupportedError("base_kernel must have lengthscale(s)")
    if hasattr(base_kernel, "lengthscale_prior"):
        raise UnsupportedError("base_kernel must not specify a lengthscale prior")
    prior_helper = SaasPriorHelper(tau=tau)
    base_kernel.register_prior(
        name="inv_lengthscale_prior",
        prior=HalfCauchyPrior(torch.tensor(1.0, **tkwargs)),
        param_or_closure=prior_helper.inv_lengthscale_prior_param_or_closure,
        setting_closure=prior_helper.inv_lengthscale_prior_setting_closure,
    )
    return base_kernel

In [5]:
branin = Branin().to(**tkwargs)


def branin_emb(x):
    """x is assumed to be in [0, 1]^d; only first two entries matter"""
    lb, ub = branin.bounds
    return branin(lb + (ub - lb) * x[..., :2])



In [6]:
D = 100
# initialize train_X data to be at the middle of the domain
train_X = 0.5 * torch.ones(3, D, **tkwargs)
# perturb the first dimension of the second point
train_X[1, 0] = torch.rand(1, **tkwargs)
# perturb the second dimension of the third point
train_X[2, 1] = torch.rand(1, **tkwargs)
train_Y = branin_emb(train_X).unsqueeze(-1)
train_Y = (train_Y - train_Y.mean()) / train_Y.std()

In [7]:
base_kernel = MaternKernel(ard_num_dims=D)
add_saas_prior(base_kernel, tau=0.1)
covar_module = ScaleKernel(base_kernel)
gp = FixedNoiseGP(
    train_X=train_X,
    train_Y=train_Y,
    train_Yvar=1e-6 * torch.ones_like(train_Y),
    covar_module=covar_module,
)
mll = ExactMarginalLogLikelihood(model=gp, likelihood=gp.likelihood)
base_kernel._set_lengthscale(1e3)  # Initialize to 1e3, i.e., all being unimportant
fit_gpytorch_scipy(
    mll, bounds={"model.covar_module.base_kernel.raw_lengthscale": [0.1, 1e3]}
)

print(gp.covar_module.base_kernel.lengthscale)

tensor([[7.4440e-01, 9.0796e-01, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03,
         1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03,
         1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03,
         1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03,
         1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03,
         1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03,
         1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03,
         1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03,
         1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03,
         1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03,
         1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03,
         1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03, 1.0000e+03,
         1.0000e+03, 1.0000e

In [8]:
gp.likelihood.__dict__

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict([(3,
               <bound method Module._load_state_hook_ignore_shapes of FixedNoiseGaussianLikelihood(
                 (noise_covar): FixedGaussianNoise()
               )>)]),
 '_modules': OrderedDict([('noise_covar', FixedGaussianNoise())]),
 '_added_loss_terms': OrderedDict(),
 '_priors': OrderedDict(),
 '_constraints': OrderedDict(),
 '_strict_init': True,
 '_load_strict_shapes': True,
 'max_plate_nesting': 1,
 'second_noise_covar': None}

In [44]:
train_Y.squeeze(1)

tensor([ 0.0290,  0.9852, -1.0142], dtype=torch.float64)

In [15]:
gp(train_X).shape

<bound method TorchDistributionMixin.shape of MultivariateNormal(loc: torch.Size([3]))>

In [9]:
gp.likelihood(train_Y.squeeze(1)).__dict__

{'loc': tensor([-0.1881,  1.0807, -0.8926], dtype=torch.float64),
 'scale': tensor([0.0010, 0.0010, 0.0010], dtype=torch.float64),
 '_batch_shape': torch.Size([3]),
 '_event_shape': torch.Size([])}

In [10]:
gp.likelihood(torch.tensor([10.0, 20.0, 30.0])).__dict__

{'loc': tensor([10., 20., 30.]),
 'scale': tensor([0.0010, 0.0010, 0.0010], dtype=torch.float64),
 '_batch_shape': torch.Size([3]),
 '_event_shape': torch.Size([])}

In [11]:
gp.likelihood(train_Y.squeeze(1)).mean

tensor([-0.1881,  1.0807, -0.8926], dtype=torch.float64)

In [12]:
gp.likelihood(gp(train_X)).mean

tensor([0.0945, 0.0945, 0.0945], dtype=torch.float64,
       grad_fn=<ExpandBackward0>)

In [23]:
gp(train_X).mean

tensor([-0.0293, -0.0293, -0.0293], dtype=torch.float64,
       grad_fn=<ExpandBackward0>)

In [27]:
gp.posterior(train_X).mean

tensor([[ 0.0290],
        [ 0.9852],
        [-1.0142]], dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)

In [ ]:
hartmann = Hartmann().to(**tkwargs)


def hartmann_emb(x):
    """x is assumed to be in [0, 1]^d; only first two entries matter"""
    lb, ub = hartmann.bounds
    return hartmann(lb + (ub - lb) * x[..., :6])


# train_X = 0.5 * torch.ones(7, D, **tkwargs)
# # for i in range(6):
# #     train_X[i+1, i] = torch.rand(1, **tkwargs)

In [24]:

D = 100 # input dimension
N=50 # number of perturbing samples

# initialize train_X data to be at the middle of the domain
train_X = 0.5 * torch.ones(N+1, D, **tkwargs)

for i in range(N):
    indices = torch.randperm(D)[:D//5]
    print(indices)
    train_X[i+1, indices] = torch.rand(D//5, **tkwargs)
    # train_X[i+1, indices] = torch.ones(D//5, **tkwargs)

train_Y = hartmann_emb(train_X).unsqueeze(-1)
train_Y = (train_Y - train_Y.mean()) / train_Y.std()

tensor([78, 39, 34, 85, 72, 10, 74, 95, 47, 50, 40,  4,  8, 14, 90,  3, 17, 61,
        89, 66])
tensor([20, 33,  8, 98,  7, 87, 23, 57, 45, 18, 51, 70, 41, 74, 11, 24, 71, 62,
        99, 68])
tensor([59, 14, 61, 17, 29, 85, 62, 18, 26, 89, 68, 28, 42, 27, 39, 49,  3, 72,
        77, 11])
tensor([80, 11, 73, 94, 26, 17, 64, 61, 89, 16, 71,  3, 92, 58, 18, 95, 10, 28,
        54,  7])
tensor([27, 59, 57, 45,  7, 61,  3, 92, 51, 98, 30,  5, 89, 76, 18, 96, 86, 20,
        16, 47])
tensor([26, 55, 47, 63, 25, 34, 31, 22, 37, 58, 75, 87, 96, 70, 61, 14, 53, 12,
        35, 68])
tensor([90, 16, 18,  9, 41, 29, 84, 23, 74, 36, 79, 99, 35, 30, 96, 91, 71, 89,
        58, 67])
tensor([77, 63, 21, 55, 39, 92, 76, 56,  6,  0, 97, 30, 60, 36, 62, 45, 70, 10,
        89, 12])
tensor([91, 14,  6, 45,  4, 85, 96, 22, 95, 37, 17, 79, 10, 42, 29, 68, 67, 56,
        23, 73])
tensor([80, 82, 34,  2, 30, 93, 99, 14, 25, 67, 88, 98,  7,  8, 36, 10, 21,  4,
        52, 84])
tensor([44, 42, 83, 20, 43, 98

In [25]:
base_kernel = MaternKernel(ard_num_dims=D)
add_saas_prior(base_kernel, tau=0.1)
covar_module = ScaleKernel(base_kernel)
gp = FixedNoiseGP(
    train_X=train_X,
    train_Y=train_Y,
    train_Yvar=1e-6 * torch.ones_like(train_Y),
    covar_module=covar_module,
)
mll = ExactMarginalLogLikelihood(model=gp, likelihood=gp.likelihood)
base_kernel._set_lengthscale(1e3)  # Initialize to 1e3
fit_gpytorch_scipy(
    mll, bounds={"model.covar_module.base_kernel.raw_lengthscale": [0.1, 1e3]}
)

learned_lengthscales = gp.covar_module.base_kernel.lengthscale
print(learned_lengthscales)
print(torch.topk(learned_lengthscales, 6, largest = False))

tensor([[6.4332e+00, 2.6280e+00, 9.5749e+02, 7.4892e-01, 1.2785e+01, 9.7170e+02,
         7.7925e+02, 9.7949e+02, 9.7670e+02, 9.3122e+02, 7.7949e+02, 7.6029e+02,
         7.4440e-01, 9.6105e+02, 7.8345e+02, 9.0409e+02, 9.4696e+02, 7.4440e-01,
         9.8873e+02, 9.9227e+02, 8.5627e+02, 1.7841e+01, 9.2891e+02, 4.6126e+02,
         8.5884e+02, 9.8534e+02, 9.1189e+02, 9.2355e+02, 6.0401e+02, 7.8146e+02,
         9.9551e+02, 7.4788e+02, 9.0212e+02, 8.7871e+02, 9.1751e+02, 9.9242e+02,
         9.4734e+02, 4.4974e+01, 4.3836e+02, 9.5027e+02, 9.5991e+02, 9.7965e+02,
         8.3798e+02, 6.9584e+02, 8.8110e+02, 9.7401e+02, 9.7575e+02, 8.4065e+02,
         8.1021e+02, 8.4079e+02, 9.2534e+02, 9.9446e+02, 5.4891e+02, 9.0335e+02,
         9.9014e+02, 9.8309e+02, 7.5944e+01, 9.9562e+02, 9.8161e+02, 6.3233e+02,
         8.3698e+02, 1.5523e+01, 9.4270e+02, 8.7397e+02, 8.6735e+02, 1.9209e+01,
         9.2408e+02, 7.7692e+02, 9.3174e+02, 4.0356e+01, 6.7949e+02, 8.8590e+01,
         9.6394e+02, 7.4584e

In [15]:
fit_fully_bayesian_model_nuts(gp)

AttributeError: 'FixedNoiseGP' object has no attribute 'pyro_model'